In [ ]:
package main


/*

開發大流程

    引入套件

    定義資料格式
    生成模擬資料
    查詢單隻tuna
    增加新紀錄
    查詢所有紀錄
    變換持有人

    設定必須的 init 與 invoke 方法

    主程序

*/



In [ ]:
/*

引入套件

下方源碼，在閱讀後，進行註解

*/

In [ ]:
import (
        "bytes"
        "encoding/json"
        "fmt"
        "strconv"

        "github.com/hyperledger/fabric/core/chaincode/shim"
        sc "github.com/hyperledger/fabric/protos/peer"
)

In [ ]:
/*

定義資料格式

下方源碼，在閱讀後，進行註解

*/

In [ ]:
type SmartContract struct {

}

type Tuna struct {
        Vessel string `json:"vessel"`
        Timestamp string `json:"timestamp"`
        Location  string `json:"location"`
        Holder  string `json:"holder"`
}

In [ ]:
/*

生成模擬資料

下方源碼，在閱讀後，進行註解

*/

In [ ]:

func (s *SmartContract) initLedger(APIstub shim.ChaincodeStubInterface) sc.Response {
        tuna := []Tuna{
                Tuna{Vessel: "923F", Location: "太平洋", Timestamp: "20180701", Holder: "Miriam"},
		Tuna{Vessel: "M83T", Location: "大西洋", Timestamp: "20180701", Holder: "Dave"},
		Tuna{Vessel: "T012", Location: "印度洋", Timestamp: "20180703", Holder: "Igor"},
		Tuna{Vessel: "P490", Location: "北冰洋", Timestamp: "20180703", Holder: "Amalea"},
		Tuna{Vessel: "S439", Location: "南冰洋", Timestamp: "20180703", Holder: "Rafa"},
		Tuna{Vessel: "J205", Location: "太平洋", Timestamp: "20180704", Holder: "Shen"},
		Tuna{Vessel: "S22L", Location: "大西洋", Timestamp: "20180705", Holder: "Leila"},
		Tuna{Vessel: "EI89", Location: "印度洋", Timestamp: "20180705", Holder: "Yuan"},
		Tuna{Vessel: "129R", Location: "北冰洋", Timestamp: "20180705", Holder: "Carlo"},
		Tuna{Vessel: "49W4", Location: "南冰洋", Timestamp: "20180703", Holder: "Fatima"},
        }

        i := 0
        for i < len(tuna) {
                fmt.Println("i is ", i)
                tunaAsBytes, _ := json.Marshal(tuna[i])
                APIstub.PutState(strconv.Itoa(i+1), tunaAsBytes)
                fmt.Println("Added", tuna[i])
                i = i + 1
        }

        return shim.Success(nil)
}


In [ ]:
/*

查詢單隻tuna

下方源碼，在閱讀後，進行註解

*/

In [ ]:
func (s *SmartContract) queryTuna(APIstub shim.ChaincodeStubInterface, args []string) sc.Response {

        if len(args) != 1 {
                return shim.Error("Incorrect number of arguments. Expecting 1")
        }

        tunaAsBytes, _ := APIstub.GetState(args[0])
        if tunaAsBytes == nil {
                return shim.Error("Could not locate tuna")
        }
        return shim.Success(tunaAsBytes)
}

In [ ]:
/*

增加新紀錄

下方源碼，在閱讀後，進行註解

*/

In [ ]:
func (s *SmartContract) recordTuna(APIstub shim.ChaincodeStubInterface, args []string) sc.Response {

        if len(args) != 5 {
                return shim.Error("Incorrect number of arguments. Expecting 5")
        }

        var tuna = Tuna{ Vessel: args[1], Location: args[2], Timestamp: args[3], Holder: args[4] }

        tunaAsBytes, _ := json.Marshal(tuna)
        err := APIstub.PutState(args[0], tunaAsBytes)
        if err != nil {
                return shim.Error(fmt.Sprintf("Failed to record tuna catch: %s", args[0]))
        }

        return shim.Success(nil)
}

In [ ]:
/*

查詢所有紀錄

下方源碼，在閱讀後，進行註解

*/

In [ ]:
func (s *SmartContract) queryAllTuna(APIstub shim.ChaincodeStubInterface) sc.Response {

        startKey := "0"
        endKey := "999"

        resultsIterator, err := APIstub.GetStateByRange(startKey, endKey)
        if err != nil {
                return shim.Error(err.Error())
        }
        defer resultsIterator.Close()

        // buffer is a JSON array containing QueryResults
        var buffer bytes.Buffer
        buffer.WriteString("[")

        bArrayMemberAlreadyWritten := false
        for resultsIterator.HasNext() {
                queryResponse, err := resultsIterator.Next()
                if err != nil {
                        return shim.Error(err.Error())
                }
                // Add comma before array members,suppress it for the first array member
                if bArrayMemberAlreadyWritten == true {
                        buffer.WriteString(",")
                }
                buffer.WriteString("{\"Key\":")
                buffer.WriteString("\"")
                buffer.WriteString(queryResponse.Key)
                buffer.WriteString("\"")

                buffer.WriteString(", \"Record\":")
                // Record is a JSON object, so we write as-is
                buffer.WriteString(string(queryResponse.Value))
                buffer.WriteString("}")
                bArrayMemberAlreadyWritten = true
        }
        buffer.WriteString("]")

        fmt.Printf("- queryAllTuna:\n%s\n", buffer.String())

        return shim.Success(buffer.Bytes())
}

In [ ]:
/*

變換持有人

下方源碼，在閱讀後，進行註解

*/


In [ ]:
func (s *SmartContract) changeTunaHolder(APIstub shim.ChaincodeStubInterface, args []string) sc.Response {

        if len(args) != 2 {
                return shim.Error("Incorrect number of arguments. Expecting 2")
        }

        tunaAsBytes, _ := APIstub.GetState(args[0])
        if tunaAsBytes == nil {
                return shim.Error("Could not locate tuna")
        }
        tuna := Tuna{}

        json.Unmarshal(tunaAsBytes, &tuna)
        // Normally check that the specified argument is a valid holder of tuna
        // we are skipping this check for this example
        tuna.Holder = args[1]

        tunaAsBytes, _ = json.Marshal(tuna)
        err := APIstub.PutState(args[0], tunaAsBytes)
        if err != nil {
                return shim.Error(fmt.Sprintf("Failed to change tuna holder: %s", args[0]))
        }

        return shim.Success(nil)
}

In [ ]:
/*

主程序

下方源碼，在閱讀後，進行註解

*/


In [ ]:
func main() {

        // Create a new Smart Contract
        err := shim.Start(new(SmartContract))
        if err != nil {
                fmt.Printf("Error creating new Smart Contract: %s", err)
        }
}